In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup
import pandas as pd
import math
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
import os
import itertools
import numpy as np

In [2]:
class TextValueDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length, column):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.column = column

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        text = row['generated_text']
        label = (row[self.column] - 1)/4 # Convert value to 0-1
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }

In [3]:
# Load dataset
def load_dataset(file_path):
    if '.tsv' in file_path:
        df = pd.read_csv(file_path, sep='\t')
    else:
        df = pd.read_csv(file_path)
    return df

# Create DataLoader
def create_dataloader(df, tokenizer, max_length, batch_size, column):
    dataset = TextValueDataset(df, tokenizer, max_length, column)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [4]:
def evaluate(model, dataloader, device):
    model.eval()
    predictions, true_labels, pos_probs = [], [], []

    eval_loss = 0
    eval_loop = tqdm(dataloader, desc="Evaluating", leave=False)
    with torch.no_grad():
        for batch in eval_loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # print(input_ids, attention_mask, labels)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits
            # print(logits)
            # preds = torch.argmax(logits, dim=1)
            # print(preds)
            
            loss = outputs.loss
            eval_loss += loss.item()
            eval_loop.set_postfix(loss=loss.item())
            
            predictions.extend(logits.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    # accuracy = accuracy_score(true_labels, predictions)
    # precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
    
    mae = mean_absolute_error(true_labels, predictions)
    mse = mean_squared_error(true_labels, predictions)
    return eval_loss, np.sqrt(mse), mse, mae

In [5]:
data_dict = { 
                'model' : [],
                 'dimension' : [],
                 'RMSE' : [],
                 'MSE' : [],
                 'MAE' : []
            }

In [6]:
def get_performance(tokname, modeldir):
    max_length = 256
    batch_size = 16
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # tokenizer = AutoTokenizer.from_pretrained('google-t5/t5-small')
    tokenizer = AutoTokenizer.from_pretrained(tokname)
    test_df = load_dataset('../data/crowd-enVent-test.tsv')
    for model_name in os.listdir(modeldir):
        # model_name='./models/google-t5/suddenness_google-t5/'
        if '.ipy' in model_name:
            continue
        column = str.join('_', model_name.split('_')[:-1])
        print(column)
        test_loader = create_dataloader(test_df, tokenizer, max_length, batch_size, column)
        model = AutoModelForSequenceClassification.from_pretrained(os.path.join(modeldir, model_name))
        model = model.to(device)
        _, test_rmse, test_mse, test_mae = evaluate(model, test_loader, device)
        
        
        print(f'Model Name: {model_name}, Test RMSE: {test_rmse:.4f}, Test MSE: {test_mse:.4f}, Test MAE: {test_mae:.4f}')
        data_dict['model'].append(model_name.split('_')[-1])
        data_dict['dimension'].append(column)
        data_dict['RMSE'].append(test_rmse)
        data_dict['MSE'].append(test_mse)
        data_dict['MAE'].append(test_mae)
        # Cleanup
        model.to('cpu')
        del model
        torch.cuda.empty_cache()
    
    return data_dict

In [7]:
def roc_curve(model, app_dim, fpr, tpr, auc):
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {auc:.2f})')
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal line for random guessing
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(model + ':' + app_dim + ' Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

In [8]:
mod_tok_dict = {
    'bert-base-uncased' : 'bert-base-uncased',
    'bert-large-uncased' : 'bert-large-uncased',
    'dialogpt' : 'microsoft/DialogRPT-updown',
    'distilbert-base-uncased' : 'distilbert-base-uncased',
    'google-t5' : 'google-t5/t5-small',
    'roberta-base' : 'roberta-base',
    'roberta-large': 'FacebookAI/roberta-large',
}

for key in mod_tok_dict:
    modeldir = os.path.join('./models', key)
    data_dict = get_performance(mod_tok_dict[key], modeldir)

/home/adebnath/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


self_control


Model Name: self_control_bert-base-uncased, Test RMSE: 0.3408, Test MSE: 0.1162, Test MAE: 0.2672
urgency


Model Name: urgency_bert-base-uncased, Test RMSE: 0.3582, Test MSE: 0.1283, Test MAE: 0.2891
predict_event


Model Name: predict_event_bert-base-uncased, Test RMSE: 0.3501, Test MSE: 0.1226, Test MAE: 0.3006
chance_control


...

Model Name: social_norms_FacebookAI, Test RMSE: 0.3346, Test MSE: 0.1120, Test MAE: 0.2678
predict_conseq


Model Name: predict_conseq_FacebookAI, Test RMSE: 0.3614, Test MSE: 0.1306, Test MAE: 0.3144
other_responsblt


Model Name: other_responsblt_FacebookAI, Test RMSE: 0.3241, Test MSE: 0.1050, Test MAE: 0.2446
chance_control


Model Name: chance_control_FacebookAI, Test RMSE: 0.3923, Test MSE: 0.1539, Test MAE: 0.3369
self_control


Model Name: self_control_FacebookAI, Test RMSE: 0.3690, Test MSE: 0.1362, Test MAE: 0.3311


In [10]:
# for ix, _ in enumerate(data_dict['model']):
#     # roc_curve(data_dict['model'], data_dict['dimension'])

TypeError: roc_curve() missing 3 required positional arguments: 'fpr', 'tpr', and 'auc'

In [11]:
df = pd.DataFrame(data_dict)
df

,model,dimension,RMSE,MSE,MAE
0,bert-base-uncased,self_control,0.340837,0.116170,0.267178
1,bert-base-uncased,urgency,0.358199,0.128306,0.289085
2,bert-base-uncased,predict_event,0.350095,0.122567,0.300647
3,bert-base-uncased,chance_control,0.340582,0.115996,0.270933
4,bert-base-uncased,effort,0.330662,0.109338,0.266773
...,...,...,...,...,...
142,FacebookAI,social_norms,0.334594,0.111953,0.267847
143,FacebookAI,predict_conseq,0.361370,0.130588,0.314368
144,FacebookAI,other_responsblt,0.324101,0.105041,0.244587
145,FacebookAI,chance_control,0.392263,0.153871,0.336888


In [12]:
df[df['model'] == 'bert-large-uncased'].describe()

,RMSE,MSE,MAE
count,21.000000,21.000000,21.000000
mean,0.336824,0.114753,0.277004
std,0.036986,0.024848,0.051267
min,0.262320,0.068812,0.176911
25%,0.327346,0.107155,0.259553
50%,0.340358,0.115843,0.279807
75%,0.350693,0.122986,0.288983
max,0.411290,0.169159,0.379276


In [13]:
df[df['model'] == 'roberta-base'].describe()

,RMSE,MSE,MAE
count,21.000000,21.000000,21.000000
mean,0.321761,0.104236,0.260431
std,0.027220,0.016866,0.031110
min,0.256373,0.065727,0.178736
25%,0.313880,0.098521,0.243398
50%,0.327237,0.107084,0.274048
75%,0.344410,0.118618,0.279331
max,0.355742,0.126553,0.302349


In [14]:
df[df['model'] == 'distilbert-base-uncased'].describe()

,RMSE,MSE,MAE
count,21.000000,21.000000,21.000000
mean,0.329429,0.109161,0.267668
std,0.025866,0.016597,0.032188
min,0.274217,0.075195,0.187304
25%,0.319028,0.101779,0.250079
50%,0.339855,0.115501,0.275937
75%,0.348023,0.121120,0.293428
max,0.365246,0.133405,0.306085


In [15]:
df[df['model'] == 'google-t5'].describe()

,RMSE,MSE,MAE
count,21.000000,21.000000,21.000000
mean,0.348815,0.122223,0.295777
std,0.024044,0.016386,0.029438
min,0.296971,0.088192,0.230874
25%,0.341761,0.116801,0.291304
50%,0.351996,0.123901,0.303170
75%,0.366699,0.134468,0.314051
max,0.386958,0.149736,0.334290


In [16]:
df[df['model'] == 'bert-base-uncased'].describe()

,RMSE,MSE,MAE
count,21.000000,21.000000,21.000000
mean,0.329125,0.108963,0.265786
std,0.025916,0.016475,0.032267
min,0.267815,0.071725,0.191849
25%,0.315301,0.099415,0.257789
50%,0.337686,0.114032,0.268092
75%,0.349059,0.121842,0.289181
max,0.358199,0.128306,0.308196


In [21]:
df[df['model'] == 'microsoft'].describe()

,RMSE,MSE,MAE
count,21.000000,21.000000,21.000000
mean,0.334287,0.112225,0.273591
std,0.022386,0.014705,0.030304
min,0.288885,0.083455,0.203294
25%,0.317354,0.100713,0.264389
50%,0.332921,0.110837,0.278148
75%,0.351846,0.123795,0.296393
max,0.360667,0.130081,0.312788


In [22]:
df[df['model'] == 'FacebookAI'].describe()

,RMSE,MSE,MAE
count,21.000000,21.000000,21.000000
mean,0.348441,0.123188,0.292807
std,0.043191,0.029984,0.056277
min,0.257111,0.066106,0.160482
25%,0.324101,0.105041,0.252638
50%,0.348768,0.121639,0.288071
75%,0.381361,0.145436,0.336888
max,0.429966,0.184871,0.402199
